In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
np.random.seed(69)

import tqdm

from sklearn.decomposition import PCA

from viz_helpers import plot_w_genre

In [2]:
movie_df = pd.read_csv('data/movies_c.txt', delimiter='\t', header=None,
                       names=["Movie Id", "Movie Title", "Unknown", "Action", 
                              "Adventure", "Animation", "Childrens", "Comedy", 
                              "Crime", "Documentary", "Drama", "Fantasy", 
                              "Film-Noir", "Horror", "Musical", "Mystery", 
                              "Romance", "Sci-Fi", "Thriller", "War", 
                              "Western"])

data_df = pd.read_csv('data/data_c.txt', delimiter='\t', header=None,
                      names=["User Id", "Movie Id", "Rating"])
train_df = pd.read_csv('data/train_c.txt', delimiter='\t', header=None,
                      names=["User Id", "Movie Id", "Rating"])
test_df = pd.read_csv('data/test_c.txt', delimiter='\t', header=None,
                      names=["User Id", "Movie Id", "Rating"])

## Helper functions

In [3]:
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    return eta * (reg * Ui - Vj * (Yij - np.dot(Ui, Vj))) 

def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    return eta * (reg * Vj - Ui * (Yij - np.dot(Ui, Vj))) 

def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    Y2 = Y - np.array([[1, 1, 0]]) # Fix one-indexing
    reg_err = reg / 2 * (np.linalg.norm(U) ** 2 + np.linalg.norm(V) ** 2)
    Is, Js, Y_ij = Y2[:, 0], Y2[:, 1], Y2[:, 2]
    mse = np.sum((Y_ij - np.sum(U[Is] * V[Js], axis=1)) ** 2) / 2
    return (reg_err + mse) / Y.shape[0]


def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300,
               desc='train_loop'):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    
    U = np.random.rand(M, K) - 0.5
    V = np.random.rand(N, K) - 0.5
    old_loss = get_err(U, V, Y, reg=reg)
    
    for e in tqdm.tqdm_notebook(range(max_epochs), desc=desc):
        Y_curr = Y[np.random.permutation(Y.shape[0])]
        for (i, j, y_ij) in Y_curr:
            i, j = i-1, j-1 # Fix One-indexing
            U[i] -= grad_U(U[i], y_ij, V[j], reg, eta)
            V[j] -= grad_V(V[j], y_ij, U[i], reg, eta)
            
        new_loss = get_err(U, V, Y, reg=reg)
        if e == 0:
            loss_delta = old_loss - new_loss
        if (old_loss - new_loss) / loss_delta <= eps:
            break
        old_loss = new_loss
        
    return (U, V, get_err(U, V, Y, reg=0.0))

## Run code

In [4]:
Y = train_df.values
Y_test = test_df.values
M = train_df["User Id"].max()
N = movie_df["Movie Id"].max()

K = 20; eta = 0.03; reg=1e-3
U, V, err = train_model(M, N, K, eta, reg, Y)

In [ ]:
pca = PCA(3)
V_p = pca.fit_transform(V)
U_p = pca.transform(U)

genres = ["Horror", "Romance", "Comedy"]
n_mov = 3
plot_w_genre(genres, movie_df, V_p, n_mov=n_mov, dim_3=True)
plt.gcf().set_size_inches(18, 10)